In [17]:
# import sys
# sys.path.append('../')
from comparisonframe import ComparisonFrame

## Usage examples

The examples contain: 
1. creating validation set and saving it to be reused
2. comparing newly generated data with expected results 
3. recording test statuses
4. reseting statuses, flushing records and comparison results

### 1. Creating validation set

### 1.1 Initialize comparison class

In [2]:
comparer = ComparisonFrame(
    # optionally
    ## mocker default parameters
    mocker_params = {
        'file_path' : "./comparisonframe_storage",
         'persist' : True},

    ## scores to calculate
    compare_scores = ['char_count_diff',
                        'word_count_diff',
                        'line_count_diff',
                        'punctuation_diff',
                        'semantic_similarity']
)

/home/kyriosskia/miniconda3/envs/testenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### 1.2 Recording queries and expected responses (validation set)

In [3]:
comparer.record_queries(
    queries = ["Black metal", 
               "Tribulation"],
    expected_texts = ["Black metal is an extreme subgenre of heavy metal music.",
    "Tribulation are a Swedish heavy metal band from Arvika that formed in 2005."],
    metadata = {'name' : 'metal_bands'})

### 2. Comparing with expected results

#### 2.1 Initialize new comparison class

In [4]:
comparer = ComparisonFrame()

### 2.2 Show validation set

In [5]:
untested_queries = comparer.get_all_queries(
    ## optional
    metadata_filters={'name' : 'metal_bands'})
print(untested_queries)

['Black metal', 'Tribulation']


In [6]:
comparer.get_all_records()

[{'expected_text': 'Black metal is an extreme subgenre of heavy metal music.',
  'record_id': '0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8bd7b663c92f2f16e87',
  'query': 'Black metal'},
 {'expected_text': 'Tribulation are a Swedish heavy metal band from Arvika that formed in 2005.',
  'record_id': 'eecd9c2a5b25ee6053891b894157fa30372ed694763385e1ada1dc9ad8e41625',
  'query': 'Tribulation'}]

In [7]:
comparer.get_all_records_df()

,expected_text,record_id,query
0,Black metal is an extreme subgenre of heavy me...,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,Black metal
1,Tribulation are a Swedish heavy metal band fro...,eecd9c2a5b25ee6053891b894157fa30372ed694763385...,Tribulation


#### 2.3 Insert newly generated with records

In [8]:
valid_answer_query_1 = "Black metal is an extreme subgenre of heavy metal music."
very_similar_answer_query_1 = "Black metal is a subgenre of heavy metal music."
unexpected_answer_query_1 = "Black metals are beautiful and are often used in jewelry design."

In [9]:
comparer.record_runs(queries = ["Black metal"],
                     provided_texts = [valid_answer_query_1,
                                      very_similar_answer_query_1,
                                      unexpected_answer_query_1],
                    metadata={'desc' : 'definitions'})

In [10]:
df = comparer.get_all_runs_df()
df

,provided_text,timestamp,query,run_id
0,Black metal is an extreme subgenre of heavy me...,2024-08-19 23:55:02,Black metal,9b116b11b58349fa7945803013fa971f190b9914bdf9db...
1,Black metal is a subgenre of heavy metal music.,2024-08-19 23:55:02,Black metal,d041163ef6e1bf8016a2817aac5025903fedeafad57bd3...
2,Black metals are beautiful and are often used ...,2024-08-19 23:55:02,Black metal,51b73dc0de1172717a3638155a95d0e13b768a23ddd07c...


### 2.4 Comparing runs with records

In [11]:
comparer.compare_runs_with_records()

In [12]:
comparer.get_all_run_scores_df()

,provided_text,timestamp,query,run_id,record_id,char_count_diff,word_count_diff,line_count_diff,punctuation_diff,semantic_similarity,comparison_id
0,Black metal is an extreme subgenre of heavy me...,2024-08-19 23:55:06,Black metal,9b116b11b58349fa7945803013fa971f190b9914bdf9db...,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,0,0,0,0,1.000000,30015aca1e9728ac448f679071c6412013c4aecc41636a...
1,Black metal is a subgenre of heavy metal music.,2024-08-19 23:55:06,Black metal,d041163ef6e1bf8016a2817aac5025903fedeafad57bd3...,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,9,1,0,0,0.985985,b2a05016bd5899e9932e66797dac9eedfc9162c5078c81...
2,Black metals are beautiful and are often used ...,2024-08-19 23:55:06,Black metal,51b73dc0de1172717a3638155a95d0e13b768a23ddd07c...,0cc157453395b440f36d1a1aee24aa76a03f5f9ab0a7a8...,8,1,0,0,0.494053,0469be6671431d1961113c7d153b28314b3e35703dba4a...


### 2.5 Calculate aggregate scores

In [13]:
comparer.calculate_aggr_scores()

In [14]:
comparer.get_all_aggr_scores_df()

""
